# Matrix Factorization - Break Down

In [1]:
# Import standard libraries
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
from collections import Counter
from time import sleep

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

## 1. Intuitions and Workflow

Firstly, we have a set $U$ of users, and a set $D$ of items.<br/>Let $R$ of size $|U| \times |D|$ be the matrix that contains all the ratings that the users have assigned to the items.<br/>We assume that we would like to discover $K$ latent features.<br/>We must then find two matrics matrices P (of size $|U| \times |K|$) and Q (of size  $|D| \times |K|$) such that their product apprioximates $|R|$:

$$\mathbf{R} \approx \mathbf{P} \times \mathbf{Q}^T = \hat{\mathbf{R}}$$

Each row of $P$ would represent the strength of the associations between **a user and the features**.<br/>Each row of $Q$ would represent the strength of the associations between **an item and the features**.<br/>To get the prediction of a rating of an item $d_j$ by $u_i$, we can calculate the dot product of their vectors:

$$\hat{r}_{ij} = p_i^T q_j = \sum_{k=1}^k{p_{ik} q_{kj}}$$

Now, we have to find a way to obtain $P$ and $Q$. One way to approach this problem is the first intialize the two matrices with some values, calculate how different their product is to $R$, and then try to minimize this difference iteratively through **gradient descent**. The difference here, usually called the error between the estimated rating and the real rating, can be calculated by the following equation for each user-item pair:

$$e_{ij}^2 = (r_{ij} - \hat{r}_{ij})^2 = (r_{ij} - \sum_{k=1}^K{p_{ik}q_{kj}})^2$$

**Squared error** is considered here. To minimize the error, we have to know in which direction we have to modify the values of $p_{ik}$ and $q_{kj}$. We know this by getting the gradient at the current valuesby differentiating the above equation with respect to $p_{ik}$ and $q_{kj}$ separately:

$$\frac{\partial}{\partial p_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(q_{kj}) = -2 e_{ij} q_{kj}$$

$$\frac{\partial}{\partial q_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(p_{ik}) = -2 e_{ij} p_{ik}$$

The update rules for the values will thus be:

$$p’_{ik} = p_{ik} + \alpha \frac{\partial}{\partial p_{ik}}e_{ij}^2 = p_{ik} + 2\alpha e_{ij} q_{kj}$$

$$q’_{kj} = q_{kj} + \alpha \frac{\partial}{\partial q_{kj}}e_{ij}^2 = q_{kj} + 2\alpha e_{ij} p_{ik}$$

$\mathbf{\alpha}$ is the learning rate, which should be small (say 0.0002) to avoid the event of oscillating around the minimum.

**Note:** If we find two matrices $\mathbf{P}$ and $\mathbf{Q}$ such that $\mathbf{P} \times \mathbf{Q}$ approximates $\mathbf{R}$, isn’t that our predictions of all the unseen ratings will be zeros? In fact, we are **not** really trying to come up with $\mathbf{P}$ and $\mathbf{Q}$ such that we can reproduce $\mathbf{R}$ exactly. Instead, **we will only try to minimise the errors of the observed user-item pairs**. In other words, if we let $T$ be a set of tuples, each of which is in the form of $(u_i, d_j, r_{ij})$, such that $T$ contains all the observed user-item pairs together with the associated ratings, we are only trying to minimise every $e_{ij}$ for $(u_i, d_j, r_{ij}) \in T$. (In other words, $T$ is our set of training data.) As for the rest of the unknowns, we will be able to determine their values once the associations between the users, items and features have been learnt.

Using the above update rules, we can then iteratively perform the operation until the error converges to its minimum. We can check the overall error as calculated using the following equation and determine when we should stop the process.

$$E = \sum_{(u_i, d_j, r_{ij}) \in T}{e_{ij}} = \sum_{(u_i,d_j,r_{ij}) \in T}{(r_{ij} - \sum_{k=1}^K{p_{ik}q_{kj}})^2}$$

### Regularization

Regularization avoids overfitting. It can be done by adding a $\beta$ parameter to the square error:

$$e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{p_{ik}q_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||P||^2 + ||Q||^2)}$$

The $\beta$ parameter controls the **magnitudes** of the user-feature and item-feature vectors such that $P$ and $Q$ would give predictions of $R$ that are not too large. In practice, $\beta$ is set to some values in the order of 0.02. The new update rules for this squared error can be obtained by a procedure similar to the one described above. The new update rules are as follows: 

$$p’_{ik} = p_{ik} + \alpha \frac{\partial}{\partial p_{ik}}e_{ij}^2 = p_{ik} + \alpha(2 e_{ij} q_{kj} - \beta p_{ik} )$$

$$q’_{kj} = q_{kj} + \alpha \frac{\partial}{\partial q_{kj}}e_{ij}^2 = q_{kj} + \alpha(2 e_{ij} p_{ik} - \beta q_{kj} )$$


### Biases

Adding biases will better model how a particular user rates items. E.g. A person who is a big movie fan may rate movies more strictly and thus give lower ratings. Biases can be added to the prediction rating as follows:

$$\hat{r}_{ij} = b + bu_i + bd_j + \sum_{k=1}^k{p_{ik} q_{kj}}$$

$\hat{r}_{ij}$ - prediction<br/>
$b$ - global bias (mean of all ratings not including missing ratings)<br/>
$bu_i$ - bias for user $i$<br/>
$bd_j$ - bias for item $j$<br/>

We can derive the update rules for user and item biases as follows:

$$bu’_i = bu_i + \alpha \times (e_{ij} - \beta bu_i)$$

$$bd’_j = bd_j + \alpha \times (e_{ij} - \beta bd_j)$$


Source: http://www.albertauyeung.com/post/python-matrix-factorization/


### 1.1. Create User - Item Ratings Matrix (R)

In [2]:
ratings = pd.DataFrame([[5, 3, 0, 1],
                        [4, 0, 0, 1],
                        [1, 1, 0, 5],
                        [1, 0, 0, 4],
                        [0, 1, 5, 4]], 
                       index=['U1','U2','U3','U4','U5'],
                       columns=['D1','D2','D3','D4'])
ratings


,D1,D2,D3,D4
U1,5,3,0,1
U2,4,0,0,1
U3,1,1,0,5
U4,1,0,0,4
U5,0,1,5,4


In [3]:
R = ratings.values
R

array([[5, 3, 0, 1],
       [4, 0, 0, 1],
       [1, 1, 0, 5],
       [1, 0, 0, 4],
       [0, 1, 5, 4]])

In [3]:
num_users, num_items = R.shape

print('num_users:', num_users)
print('num_items:', num_items)

num_users: 5
num_items: 4


### 1.2. Decide the Number of Latent Features (K)

In [4]:
K = 2

### 1.3. Initialize User Latent Feature Matrix (P) and Item Latent Feature Matrix (Q)

In [7]:
# Initialize with random values
P = np.random.normal(scale=1./K, size=(num_users, K))
print(P.shape)
P

(5, 2)


array([[ 0.17365756, -0.07654741],
       [ 0.1766468 ,  0.16675567],
       [-0.62707613,  0.3314999 ],
       [-0.90082714, -0.52598719],
       [-0.41930046,  0.45092763]])

In [8]:
# Initialize with random values
Q = np.random.normal(scale=1./K, size=(num_items, K))
print(Q.shape)
Q

(4, 2)


array([[-0.79468844,  0.092493  ],
       [ 0.17407833,  0.27195144],
       [-0.28755566,  0.26333669],
       [-0.77220423, -0.40805792]])

In [12]:
# Check if the product of the 2 matrices will be the same shape as R
print(np.dot(P,Q.T).shape == R.shape)

True


### 1.4. Initialize Biases

In [28]:
# Initialize biases
b_u = np.zeros(num_users)
b_i = np.zeros(num_items)
b = np.mean(R[np.where(R != 0)])

print(b_u)
print(b_i)
print(b)

[0. 0. 0. 0. 0.]
[0. 0. 0. 0.]
2.769230769230769


### 1.5. Make Predictions

In [32]:
# Prediction of the full matrix given our biases above
full_matrix = b + b_u[:,np.newaxis] + b_i[np.newaxis:,] + np.dot(P, Q.T)
full_matrix

array([[2.62414701, 2.77864361, 2.69913681, 2.66636744],
       [2.64427533, 2.84533059, 2.76234787, 2.56477739],
       [3.29822234, 2.75022228, 3.03684615, 3.11819045],
       [3.43645756, 2.46937331, 2.88975699, 3.67948654],
       [3.14415165, 2.81887006, 3.00854878, 2.90901177]])

In [33]:
# Visually compare to R
R

array([[5, 3, 0, 1],
       [4, 0, 0, 1],
       [1, 1, 0, 5],
       [1, 0, 0, 4],
       [0, 1, 5, 4]])

### 1.6. See How Different the PxQ is from R (Calculate Mean Squared Error)

Mean Squared Error (MSE) is calculated by the following formula:

$$e_{ij}^2 = (r_{ij} - \hat{r}_{ij})^2 = (r_{ij} - \sum_{k=1}^K{p_{ik}q_{kj}})^2$$

In [15]:
# Indices in R that are non-zero
xs, ys = R.nonzero()

print(xs)
print(ys)

[0 0 0 1 1 2 2 2 3 3 4 4 4]
[0 1 3 0 3 0 1 3 0 3 1 2 3]


In [34]:
# Calculate error
error = 0
for x, y in zip(xs, ys):
    error += (R[x,y] - full_matrix[x,y])**2
    

In [35]:
print('mean squared error:', error)
print('root mean squared error:', np.sqrt(error))

mean squared error: 39.14676121360276
root mean squared error: 6.2567372658281535


### 1.7. Reduce this Error with Gradient Descent (1 Iteration for now)

Updating latent feature matrices $P$ and $Q$:

$$p’_{ik} = p_{ik} + \alpha \frac{\partial}{\partial p_{ik}}e_{ij}^2 = p_{ik} + \alpha(2 e_{ij} q_{kj} - \beta p_{ik} )$$

$$q’_{kj} = q_{kj} + \alpha \frac{\partial}{\partial q_{kj}}e_{ij}^2 = q_{kj} + \alpha(2 e_{ij} p_{ik} - \beta q_{kj} )$$

Updating biases $bu_i$ and $bd_j$:

$$bu’_i = bu_i + \alpha \times (e_{ij} - \beta bu_i)$$

$$bd’_j = bd_j + \alpha \times (e_{ij} - \beta bd_j)$$

In [29]:
# Training samples - tuples of user, item, rating if rating is not 0
samples = [(u, i, R[u,i]) for u in range(num_users) for i in range(num_items) if R[u,i]>0]
samples

[(0, 0, 5),
 (0, 1, 3),
 (0, 3, 1),
 (1, 0, 4),
 (1, 3, 1),
 (2, 0, 1),
 (2, 1, 1),
 (2, 3, 5),
 (3, 0, 1),
 (3, 3, 4),
 (4, 1, 1),
 (4, 2, 5),
 (4, 3, 4)]

In [39]:
# alpha - learning rate
# beta - regularization parameter
alpha = 0.1
beta = 0.01

# Compute error, use update rule on biases, then update latent feature matrices
for u, i, r in samples:
    
    # For each training sample, compute prediction and error
    # For each rating prediction, we will add bias, user bias and item bias
    prediction = b + b_u[u] + b_i[i] + np.dot(P[u,:], Q[i,:])
    e = (r - prediction)
    
    # Update biases
    b_u[u] += alpha * (e - beta * b_u[u])
    b_i[i] += alpha * (e - beta * b_i[i])
    
    # Update user and item latent feature matrices
    P[u, :] += alpha * (e * Q[i, :] - beta * P[u, :])
    Q[i, :] += alpha * (e * P[u, :] - beta * Q[i, :])
    

In [40]:
# See how our biases have been updated
print('updated user biases:', b_u)
print('updated item biases:', b_i)

updated user biases: [ 0.03403453 -0.04151773 -0.14011109 -0.18565306  0.13877416]
updated item biases: [-0.16077853 -0.31257136  0.21525295  0.06332075]


In [41]:
# See how our latent feature matrices P and Q have been updated
print('updated P - user latent features matrix')
print(P)
print()
print('updated Q - item latent features matrix')
print(Q)

updated P - user latent features matrix
[[ 0.14253307  0.0292047 ]
 [ 0.20986517  0.2388028 ]
 [-0.67282281  0.13938515]
 [-0.79384293 -0.55382042]
 [-0.6098567   0.4307764 ]]

updated Q - item latent features matrix
[[-0.49478692  0.14824369]
 [ 0.31509836  0.1641975 ]
 [-0.39907462  0.36407304]
 [-1.11360151 -0.40617986]]


### 1.8. Re-calculate Mean Squared Error

In [42]:
# Prediction of the full matrix given our biases above
full_matrix_2 = b + b_u[:,np.newaxis] + b_i[np.newaxis:,] + np.dot(P, Q.T)
full_matrix_2

array([[2.57629268, 2.54040122, 2.97226956, 2.69599864],
       [2.49849697, 2.52048067, 2.94615579, 2.46033073],
       [2.82190804, 2.12742965, 3.1636255 , 3.38508149],
       [2.73348189, 1.92993182, 2.91400213, 3.75587385],
       [3.1128354 , 2.47400113, 3.52347028, 3.47549033]])

In [43]:
# Indices in R that are non-zero
xs, ys = R.nonzero()

print(xs)
print(ys)

[0 0 0 1 1 2 2 2 3 3 4 4 4]
[0 1 3 0 3 0 1 3 0 3 1 2 3]


In [44]:
# Calculate error
error_2 = 0
for x, y in zip(xs, ys):
    error_2 += (R[x,y] - full_matrix_2[x,y])**2
    

In [49]:
# See the effect of gradient descent on our mse and rmse after 1 iteration
print('0 Iterations')
print('mse:', error)
print('rmse:', np.sqrt(error))
print()
print('1 Iterations')
print('mse:', error_2)
print('rmse:', np.sqrt(error_2))

0 Iterations
mse: 39.14676121360276
rmse: 6.2567372658281535

1 Iterations
mse: 28.23997195578064
rmse: 5.314129463588617


In [50]:
# See the effect of gradient descent on our full matrix after 1 iteration
print('Full Matrix at 0 Iterations')
print(full_matrix)
print()
print('Full Matrix at 1 Iterations')
print(full_matrix_2)

Full Matrix at 0 Iterations
[[2.62414701 2.77864361 2.69913681 2.66636744]
 [2.64427533 2.84533059 2.76234787 2.56477739]
 [3.29822234 2.75022228 3.03684615 3.11819045]
 [3.43645756 2.46937331 2.88975699 3.67948654]
 [3.14415165 2.81887006 3.00854878 2.90901177]]

Full Matrix at 1 Iterations
[[2.57629268 2.54040122 2.97226956 2.69599864]
 [2.49849697 2.52048067 2.94615579 2.46033073]
 [2.82190804 2.12742965 3.1636255  3.38508149]
 [2.73348189 1.92993182 2.91400213 3.75587385]
 [3.1128354  2.47400113 3.52347028 3.47549033]]


**Observations:** In section 1, we have seen how gradient descent has effectively decreased the mean squared error of our predictions after 1 iteration. It can be imagined that this process can be repeated for a number of iterations such that we get the lowers RMSE possible. In order to implement this iterative process, we must first consolidate the method above into functions and further into a class.

## 2. Consolidating the Process

The first function, train() is the main function in which the other helper functions will fall into.

In [120]:
def train(R, K=2, iterations=10, alpha=0.1, beta=0.01):
    '''
    Function that runs the training process. Steps:
    
    1. Initialize user and item latent features matrices.
    2. Initialize global, user and item biases.
    3. Implement gradient descent.
    4. Calculate RMSE at each interation.
    '''    
    
    # Initialize P and Q with random values - CLASS ATTR
    num_users, num_items = R.shape
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    # Initialize biases - CLASS ATTR
    b_u = np.zeros(num_users)
    b_i = np.zeros(num_items)
    b = np.mean(R[np.where(R != 0)])

    # Training samples - tuples of user, item, rating if rating is not 0
    samples = [(u, i, R[u,i]) for u in range(num_users) for i in range(num_items) if R[u,i]>0]

    # Stochastic gradient descent for given number of iterations
    training_process = []
    for i in range(iterations):
        # Shuffles the training samples
        np.random.shuffle(samples)
        # Gradient descent - at each iteration, biases and latent feature matrices updated
        gradient_descent(P = P, Q = Q, b = b, b_i = b_i, b_u = b_u, 
                         alpha = alpha, beta = beta, samples = samples)
        # RMSE calculated
        root_mse = rmse(R, P = P, Q = Q, b = b, b_u = b_u, b_i = b_i)
        training_process.append((i, root_mse))
        # Print every 10 iterations
        if (i+1) % 10 == 0:
            print("Iteration: %d ; error = %.4f" % (i+1, root_mse))
    
    # Store latent features matrices in a tuple - CLASS ATTRR        
    latent_ft_mats = (P, Q)
    
    # Store biases in a tuple - CLASS ATTR
    biases = (b_u, b_i, b)
    
    # Full Matrix - CLASS ATTR
    final_mat = full_matrix(P = P, Q = Q, b = b, b_u = b_u, b_i = b_i)

    return training_process, latent_ft_mats, biases, final_mat

In [121]:
def gradient_descent(P = P, Q = Q, b = b, b_i = b_i, b_u = b_u, 
                     alpha = alpha, beta = beta, samples = samples):
    '''
    Function that runs gradient descent. Steps:
    
    1. Makes a prediction for each training rating.
    2. Get the error for each prediction.
    3. Update biases based on error, learning rate (alpha) and regularization parameter (beta).
    4. Update latent feature matrices based on the same.
    '''

    # Compute error, use update rule on biases, then update latent feature matrices
    for u, i, r in samples:

        # For each training sample, compute prediction and error
        # For each rating prediction, we will add bias, user bias and item bias
        prediction = b + b_u[u] + b_i[i] + np.dot(P[u,:], Q[i,:])
        e = (r - prediction)

        # Update biases
        b_u[u] += alpha * (e - beta * b_u[u])
        b_i[i] += alpha * (e - beta * b_i[i])

        # Update user and item latent feature matrices
        P[u, :] += alpha * (e * Q[i, :] - beta * P[u, :])
        Q[i, :] += alpha * (e * P[u, :] - beta * Q[i, :])


In [122]:
def full_matrix(P = P, Q = Q, b = b, b_u = b_u, b_i = b_i):
    '''
    Function that computes the full matrix based on updated P, Q and biases. 
    '''
    return b + b_u[:,np.newaxis] + b_i[np.newaxis:,] + np.dot(P, Q.T)

In [123]:
def rmse(R, P = P, Q = Q, b = b, b_u = b_u, b_i = b_i):
    '''
    Function that calculates the total root mean squared error.
    '''    
    
    # Predict full matrix
    predicted = full_matrix(b = b, b_u = b_u, b_i = b_i, P = P, Q = Q)
    
    # Calculate error for ratings that are not zero (this is the only way to score)
    xs, ys = R.nonzero()
    error = 0
    for x, y in zip(xs, ys):
        error += (R[x,y] - predicted[x,y])**2
        
    return np.sqrt(error)

### 2.1. Testing out our functions

In [131]:
ratings = pd.DataFrame([[5, 3, 0, 1],
                        [4, 0, 0, 1],
                        [1, 1, 0, 5],
                        [1, 0, 0, 4],
                        [0, 1, 5, 4]], 
                       index=['U1','U2','U3','U4','U5'],
                       columns=['D1','D2','D3','D4'])

R = ratings.values
R

array([[5, 3, 0, 1],
       [4, 0, 0, 1],
       [1, 1, 0, 5],
       [1, 0, 0, 4],
       [0, 1, 5, 4]])

In [132]:
train_process, latent_ft_mats, biases, final_mat = train(R, K=2, iterations=1000, alpha=0.1, beta=0.01)

Iteration: 10 ; error = 0.4125
Iteration: 20 ; error = 0.0898
Iteration: 30 ; error = 0.0439
Iteration: 40 ; error = 0.0379
Iteration: 50 ; error = 0.0348
Iteration: 60 ; error = 0.0372
Iteration: 70 ; error = 0.0378
Iteration: 80 ; error = 0.0355
Iteration: 90 ; error = 0.0398
Iteration: 100 ; error = 0.0374
Iteration: 110 ; error = 0.0366
Iteration: 120 ; error = 0.0370
Iteration: 130 ; error = 0.0335
Iteration: 140 ; error = 0.0372
Iteration: 150 ; error = 0.0382
Iteration: 160 ; error = 0.0354
Iteration: 170 ; error = 0.0353
Iteration: 180 ; error = 0.0357
Iteration: 190 ; error = 0.0360
Iteration: 200 ; error = 0.0359
Iteration: 210 ; error = 0.0377
Iteration: 220 ; error = 0.0381
Iteration: 230 ; error = 0.0351
Iteration: 240 ; error = 0.0341
Iteration: 250 ; error = 0.0368
Iteration: 260 ; error = 0.0387
Iteration: 270 ; error = 0.0362
Iteration: 280 ; error = 0.0335
Iteration: 290 ; error = 0.0361
Iteration: 300 ; error = 0.0386
Iteration: 310 ; error = 0.0355
Iteration: 320 ; 

In [133]:
train_process


[(0, 5.256810847147864),
 (1, 4.3178455976145615),
 (2, 3.094749106077734),
 (3, 2.1903984115173993),
 (4, 1.5500027956076419),
 (5, 1.1450396849027542),
 (6, 0.8656224818734857),
 (7, 0.6581412325116689),
 (8, 0.523116883039666),
 (9, 0.41247411284436175),
 (10, 0.3355972327013934),
 (11, 0.27468420405615557),
 (12, 0.23691368145865768),
 (13, 0.20020904457535302),
 (14, 0.1675035382516854),
 (15, 0.15019777038000817),
 (16, 0.13384031648996222),
 (17, 0.11303552631290296),
 (18, 0.10061168758862817),
 (19, 0.08983765012907471),
 (20, 0.08117510885476409),
 (21, 0.07573609063117957),
 (22, 0.06869594155242598),
 (23, 0.06170013384836862),
 (24, 0.05690406177455608),
 (25, 0.05365989904064138),
 (26, 0.05014634144563248),
 (27, 0.04531357712198612),
 (28, 0.046839157553227026),
 (29, 0.04393986919337992),
 (30, 0.04509148178117489),
 (31, 0.04081998904580534),
 (32, 0.04043774317856896),
 (33, 0.040647739922707544),
 (34, 0.03868827235332857),
 (35, 0.039543599186509536),
 (36, 0.04046

In [134]:
# See how our latent feature matrices P and Q have been updated
print('updated P - user latent features matrix')
print(latent_ft_mats[0])
print()
print('updated Q - item latent features matrix')
print(latent_ft_mats[1])

updated P - user latent features matrix
[[ 1.35776956 -0.45330127]
 [ 1.01576292 -0.35030624]
 [-1.42531773  0.2796585 ]
 [-1.07189331  0.21264541]
 [-0.99190847 -0.27190721]]

updated Q - item latent features matrix
[[ 1.34705803 -0.38076503]
 [ 0.7820832   0.17359821]
 [-1.02099192 -0.41646427]
 [-1.3208605   0.32690635]]


In [135]:
# See how our biases have been updated
print('updated user biases:', biases[0])
print('updated item biases:', biases[1])

updated user biases: [ 0.1229443  -0.36421252  0.17645183 -0.3230978  -0.05691351]
updated item biases: [ 0.0887355  -0.87447299  1.14972717  0.06107305]


In [136]:
# See final matrix
print('Final Matrix')
final_mat

Final Matrix


array([[4.98250623, 3.00089856, 2.84441428, 1.01163688],
       [3.99542972, 2.26414385, 2.66354972, 1.00989284],
       [1.00794822, 1.00504077, 5.43417988, 4.98082369],
       [1.00999804, 0.77026509, 4.60169533, 3.99254269],
       [1.56842724, 1.01488671, 4.98801459, 3.99467483]])

## 3. Matrix Factorization Class

In [110]:
import numpy as np

class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.
        
        Arguments:
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Compute prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Compute the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [111]:
ratings = pd.DataFrame([[5, 3, 0, 1],
                        [4, 0, 0, 1],
                        [1, 1, 0, 5],
                        [1, 0, 0, 4],
                        [0, 1, 5, 4]],
                       index=['U1','U2','U3','U4','U5'],
                       columns=['D1','D2','D3','D4'])

# Define R
R = ratings.values

# Initialize the matrix factorization calss
mf = MF(R, K=2, alpha=0.1, beta=0.01, iterations=1000)

In [113]:
# Train model
training_procc = mf.train()

Iteration: 10 ; error = 0.4823
Iteration: 20 ; error = 0.1155
Iteration: 30 ; error = 0.0459
Iteration: 40 ; error = 0.0396
Iteration: 50 ; error = 0.0386
Iteration: 60 ; error = 0.0365
Iteration: 70 ; error = 0.0371
Iteration: 80 ; error = 0.0388
Iteration: 90 ; error = 0.0399
Iteration: 100 ; error = 0.0372
Iteration: 110 ; error = 0.0404
Iteration: 120 ; error = 0.0366
Iteration: 130 ; error = 0.0356
Iteration: 140 ; error = 0.0400
Iteration: 150 ; error = 0.0388
Iteration: 160 ; error = 0.0355
Iteration: 170 ; error = 0.0358
Iteration: 180 ; error = 0.0390
Iteration: 190 ; error = 0.0399
Iteration: 200 ; error = 0.0389
Iteration: 210 ; error = 0.0372
Iteration: 220 ; error = 0.0357
Iteration: 230 ; error = 0.0384
Iteration: 240 ; error = 0.0340
Iteration: 250 ; error = 0.0355
Iteration: 260 ; error = 0.0367
Iteration: 270 ; error = 0.0404
Iteration: 280 ; error = 0.0365
Iteration: 290 ; error = 0.0374
Iteration: 300 ; error = 0.0393
Iteration: 310 ; error = 0.0374
Iteration: 320 ; 

In [114]:
training_procc

[(0, 5.842055996781901),
 (1, 5.532194253061392),
 (2, 5.111687202224677),
 (3, 4.425241314646567),
 (4, 3.31694638426138),
 (5, 2.071262096368515),
 (6, 1.2139332064395731),
 (7, 0.8121552093242488),
 (8, 0.6024508320263808),
 (9, 0.4823053884330457),
 (10, 0.3997455561075574),
 (11, 0.340836959253712),
 (12, 0.2916102203172867),
 (13, 0.24972136773010975),
 (14, 0.22210346102425235),
 (15, 0.1897232724065017),
 (16, 0.17154307578497605),
 (17, 0.15597587559793633),
 (18, 0.12892681292752298),
 (19, 0.11547098092787549),
 (20, 0.10482865292566174),
 (21, 0.09272199213934385),
 (22, 0.084290100184207),
 (23, 0.07320989103187436),
 (24, 0.07335568289435004),
 (25, 0.06384177830143835),
 (26, 0.056794122396598846),
 (27, 0.05319766504907028),
 (28, 0.05067649813610341),
 (29, 0.045885378660585674),
 (30, 0.0451302492018159),
 (31, 0.04488865443589238),
 (32, 0.04463818289596638),
 (33, 0.040975328894948274),
 (34, 0.040451151965941255),
 (35, 0.04062318921782814),
 (36, 0.041242958592428

In [118]:
# For reference
print(R)

mf.get_rating(0, 1)

[[5 3 0 1]
 [4 0 0 1]
 [1 1 0 5]
 [1 0 0 4]
 [0 1 5 4]]


3.0009324747956208

In [119]:
mf.full_matrix()

array([[4.98194498, 3.00093247, 2.3784337 , 1.01242681],
       [3.99486291, 2.24952314, 2.32799669, 1.01166204],
       [1.00799302, 1.00624237, 5.84265511, 4.98357297],
       [1.01021892, 0.76830632, 4.90953484, 3.99582105],
       [1.39315713, 1.01664805, 4.98871382, 3.99816089]])